In [ ]:
import pandas as pd

Loading data.

In [ ]:
df = pd.read_excel("data/CTRadiomics+Clinical.xlsx", header=2)
# df = pd.read_excel("data/synthetic_data.xlsx", header=2)

In [ ]:
df = df.drop(columns=["Patients"])

In [ ]:
df

If either _initial surgery date_ or _reoccurence surgery date_ is missing --> cannot define the _reoccurence_.

In [ ]:
# Drop from 'df_init' if initial surgery date or reoccurence surgery date is missing
df = df[df["Initial surgery date"].notna() & df["Reoccurence surgery date"].notna()]

In [ ]:
df

Standardizing the data fields format.

In [ ]:
df.loc[:, "Initial surgery date"] = df["Initial surgery date"].apply(lambda x: str(x).replace(",", "."))
df.loc[:, "Reoccurence surgery date"] = df["Reoccurence surgery date"].apply(lambda x: str(x).replace(",", "."))

df.loc[:, "Initial surgery date"] = pd.to_datetime(df["Initial surgery date"], dayfirst=True)
df.loc[:, "Reoccurence surgery date"] = pd.to_datetime(df["Reoccurence surgery date"], dayfirst=True)

Defining reocurrence after _t_ years based on _initial surgery date_ and _reocurrence surgery date_.

In [ ]:
df.loc[:, "Recurrence (after 1 year)"] = df.apply(lambda x: "Y" if (x["Reoccurence surgery date"] - x["Initial surgery date"]).days < 365 else "N", axis=1)
df.loc[:, "Recurrence (after 3 years)"] = df.apply(lambda x: "Y" if (x["Reoccurence surgery date"] - x["Initial surgery date"]).days < 1095 else "N", axis=1)
df.loc[:, "Recurrence (after 5 years)"] = df.apply(lambda x: "Y" if (x["Reoccurence surgery date"] - x["Initial surgery date"]).days < 1825 else "N", axis=1)

In [ ]:
df.drop(columns=["Initial surgery date", "Reoccurence surgery date"], inplace=True)

(Temporary) Manually selecting the training data.

In [ ]:
# df = df.iloc[:5, :].copy()

In [ ]:
df.info()

Separating the _clinical_ and _imagistic_ features and storing their corresponding names for later use.

In [ ]:
clinical_cols = df.columns[:25].tolist()
imagistic_cols = df.columns[25:].tolist()

print(f"Clinical cols: \t{clinical_cols}")
print(f"Imagistic cols: {imagistic_cols}")

Defining the possible target features (which we want to predict).

In [ ]:
cols_possible_targets = ["Recurrence (after 1 year)", "Recurrence (after 3 years)", "Recurrence (after 5 years)"]
col_target = cols_possible_targets[1]

In [ ]:
df = df.drop(columns=["Medic", "PreOP Imagistic", "PostOP", "Diagnostic", "Category", "Recurrence", "Relation To Brain", "LocationCategory", "Skull Base", "Parasagital", "Side", "Bilateral", "ICH", "Age At Reoccurence Operation"])

In [ ]:
df

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

Visualizing the distribution of the samples by a selected _recurrence_ target feature.

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
ax.set_title(f"{col_target} distribution")
df[col_target].value_counts().plot.bar(ax=ax)

for i, v in enumerate(df[col_target].value_counts()):
    ax.text(i, v/2 + 0.01, v, ha="center")

fig.show()

Inspection of the percentage of missing data per _feature_.

In [ ]:
fig, ax = plt.subplots(figsize=(100, 10))
ax.set_title("Percentage of missing values")
df_temp = (df.isna().sum() / len(df)).sort_values(ascending=False)
df_temp.plot.bar(ax=ax)
ax.set_xticklabels(ax.get_xticklabels())
# Annotate percentage (rotated)
for i, v in enumerate(df_temp):
    ax.text(i, v + 0.01, f"{v:.2%}", ha="center", rotation=45)
fig.show()

Mapping of the numerical values that initially should have been stored as categorical.

In [ ]:
import numpy as np

df["Headache"] = df["Headache"].map({0.0: "N", 1.0: "Y"})
df["Visaul impairment"] = df["Visaul impairment"].map({0.0: "N", 1.0: "Y"})
df["Motor deficits"] = df["Motor deficits"].map({0.0: "N", 1.0: "Y"})
df["Sensibility deficits"] = df["Sensibility deficits"].map({0.0: "N", 1.0: "Y"})

df["Grade"] = df["Grade"].astype("str").replace('nan', np.nan)

Elimination of the features that are not well represented.

In [ ]:
df = df.drop(columns=df_temp[df_temp > 0.99].index)

Separation of columns by data kind:
* Numerical
* Categorical
* Categorical (ordinal)

In [ ]:
# Selection of columns by kind
numerical_cols = df.select_dtypes(include=['number']).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).drop(cols_possible_targets, axis=1).columns.tolist()
ordinal_cols = { 
    "Resection Grade": [
        "Simpson I.",
        "Simpson II.",
        "Simpson III.",
        "Simpson IV.",
    ],
    "Grade": [
        "1.0",
        "2.0",
        "3.0"
    ]
}

# Remove any ordinal feature names from the categorical feature list
categorical_cols = [col for col in categorical_cols if col not in ordinal_cols.keys()]

Separation of features by data kind and by clinical or imagistic.

In [ ]:
clinical_numerical_cols = [col for col in numerical_cols if col in clinical_cols]
clinical_categorical_cols = [col for col in categorical_cols if col in clinical_cols]
imagistic_numerical_cols = [col for col in numerical_cols if col in imagistic_cols]
imagistic_categorical_cols = [col for col in categorical_cols if col in imagistic_cols]

print("------ Clinical features ------")
print(f"Clinical numeric features: \t{clinical_numerical_cols}")
print(f"Clinical categorical features: \t{clinical_categorical_cols}")

print("\n------ Imagistic features ------")
print(f"Imagistic numeric features: \t{imagistic_numerical_cols}")
print(f"Imagistic categorical features: {imagistic_categorical_cols}")

leftovers = [col for col in numerical_cols + categorical_cols if col not in clinical_cols + imagistic_cols]
if leftovers:
    print(f"\n⚠️ Unclassified features: {leftovers}")

In [ ]:
# Columns kept with reasonable number of observations
print(f"Numerical features: \t{numerical_cols}")
print(f"Categorical features: \t{categorical_cols}")
print(f"Ordinal features: \t{list(ordinal_cols.keys())}")

Preprocessing of the text data to eliminate format inconsistencies.

In [ ]:
# Removal of value format inconsistencies
df[list(ordinal_cols.keys())] = df[list(ordinal_cols.keys())].apply(lambda x: x.str.strip())

In [ ]:
df.drop(columns=list(set(cols_possible_targets) - {col_target}), inplace=True)

In [ ]:
df

Defining general data preprocessing pipelines (one for each distinct feature group).

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, VarianceThreshold, f_classif
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("selector_1", VarianceThreshold(threshold=0.01)),
    ("selector_2", SelectKBest(score_func=f_classif, k=15)), # Selects k-best features based on F-statistic
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(fill_value="Unknown", strategy="constant")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

ordinal_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OrdinalEncoder(categories=list(ordinal_cols.values()), handle_unknown="use_encoded_value", unknown_value=-1))
])

preprocessor = ColumnTransformer(transformers=[
    ("num_feats", numeric_transformer, numerical_cols),
    ("cat_feats", categorical_transformer, categorical_cols),
    ("ord_feats", ordinal_transformer, list(ordinal_cols.keys()))
])


In [ ]:
preprocessor

In [ ]:
# Setup predictive model
from sklearn.ensemble import RandomForestClassifier

clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(random_state=69, n_estimators=10))
])

In [ ]:
clf

In [ ]:
# Prepare data
from sklearn.model_selection import StratifiedKFold, cross_val_score

X = df.drop(columns=col_target)
y = df[col_target].astype("category").cat.codes
categories = df[col_target].astype("category").cat.categories

In [ ]:
X

In [ ]:
X.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42, stratify=y)

In [ ]:
print(f"X_train.shape: {X_train.shape} | y_train.shape: {y_train.shape}")
print(f"X_test.shape:  {X_test.shape} | y_test.shape:  {y_test.shape}")

In [ ]:
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
scores = cross_val_score(clf, X_train, y_train, cv=cv, scoring="f1_macro")

print(f"F1 scores: {scores}")
print(f"Mean F1 score: {scores.mean():.4f} +/- {scores.std():.4f}")

In [ ]:
# Train model
clf.fit(X_train, y_train)

In [ ]:
# Print all the kept features
clf["preprocessor"].get_feature_names_out().__len__()

In [ ]:
for feat in clf["preprocessor"].get_feature_names_out():
    print(feat)
    

Running on test data.

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

y_pred = clf.predict(X_test)

print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1 score: {f1_score(y_test, y_pred):.4f}")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

y_test_series = pd.Series(y_test)
y_pred_series = pd.Series(y_pred)

cm = confusion_matrix(y_test_series, y_pred_series)

fig, ax = plt.subplots(figsize=(4, 4))
sns.heatmap(cm, annot=True, cmap="YlGnBu", ax=ax, 
            xticklabels=categories, yticklabels=categories, fmt='g')
ax.set_xlabel("Predicted")
ax.set_ylabel("True")
ax.set_title(f"Confusion Matrix ({col_target})")
fig.set_dpi(100)
plt.tight_layout()
plt.show()


In [ ]:
clf.predict_proba(X_test)

In [ ]:
y_pred

In [ ]:
y_test

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import RocCurveDisplay

print("Plotting ROC Curve...")
disp = RocCurveDisplay.from_estimator(clf, X_test, y_test)
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.show()
